<a href="https://colab.research.google.com/github/hebatoallah-yehya1111/Fake_OR_Real/blob/main/Deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
%%writefile app.py
import pickle
import keras
import keras.backend as K
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import streamlit as st
df = pd.read_csv("final_DF.csv")
df.drop(['Unnamed: 0'], axis=1, inplace=True)
tags = df['Label']
encoder = LabelEncoder()
encoder.fit(tags)
def f1_metric(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  recall = true_positives / (possible_positives + K.epsilon())
  f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
  return f1_val

model = keras.models.load_model('my_model.h5', custom_objects={'f1_metric': f1_metric})
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
def predict_pipeLine(Test_name):

    tokens = tokenizer.texts_to_matrix([Test_name], mode='tfidf')
    predict_x=model.predict(np.array(tokens))
    c=np.argmax(predict_x,axis=1)
    xc = encoder.inverse_transform(c)
    print(predict_x)
    if predict_x[0][1]> 0.5 and abs(predict_x[0][1]-predict_x[0][0])>0.1 :
        return("True Name with High Confidence")
    elif predict_x[0][1]> 0.5 and abs(predict_x[0][1]-predict_x[0][0])<0.1 :
        return ("True Name with Low Confidence")
    else:
        return ("False Name")
st.set_page_config(page_title='real and fake names',page_icon=":tada:")
st.title("Real and Fake Names app")
name=st.text_input("Name")
confidence=""
if st.button('GO'):
  confidence=predict_pipeLine(name)
st.success(confidence)




Overwriting app.py


In [6]:
!streamlit run app.py & npx localtunnel --port 8501

[#.................] - fetchMetadata: sill resolveWithNewModule follow-redirect


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.148.24.212:8501

npx: installed 22 in 3.867s
your url is: https://green-coins-warn-34-148-24-212.loca.lt
2022-12-11 02:33:07.601988: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
1/1 [==============================] - 0s 110ms/step
[[0.5035634  0.53358704]]
1/1 [==============================] - 0s 39ms/step
[[0.6905689  0.30281332]]
1/1 [==============================] - 0s 38ms/step
[[0.6905689  0.30281332]]
1/1 [==============================] - 0s 42ms/step
[[0.5035634  0.53358704]]
2022-12-11 02:34:04.580 5 out of the last 5 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7ff547dce310> triggered tf.function retracing. Tracing is expensive and the excessive number of